In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
%matplotlib inline

In [19]:
#dataset = os.path.join(parentdir,'ml-latest-small/ratings.1.csv')    
#ratings = pd.read_csv(dataset, sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)

dataset = os.path.join(parentdir,'ml-latest-small/ratings_1m.dat')    
ratings = pd.read_csv(dataset, sep='::',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

C:\Users\josse\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


FileNotFoundError: File b'ml-latest-small/movies.csv' does not exist

In [7]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


,user,item,rating,timestamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


In [10]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)


In [11]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [12]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))
    

In [13]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2839,5.393953,7,1,ItemItem,West Beirut (West Beyrouth) (1998),Drama,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,923,5.310104,7,2,ItemItem,Citizen Kane (1941),Drama|Mystery,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2503,5.280088,7,3,ItemItem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,904,5.234100,7,4,ItemItem,Rear Window (1954),Mystery|Thriller,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,750,5.218964,7,5,ItemItem,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
test = all_recs_joined.loc[all_recs_joined["user"]==1]
test = test.loc[test["Algorithm"] == "ALS"]
test.reset_index()
test.iloc[90:100,:]

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1208090,3365,4.399191,1,91,ALS,"Searchers, The (1956)",Drama|Western,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1208091,1189,4.398129,1,92,ALS,"Thin Blue Line, The (1988)",Documentary,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1208092,1233,4.397450,1,93,ALS,"Boot, Das (Boat, The) (1981)",Action|Drama|War,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1208093,954,4.396120,1,94,ALS,Mr. Smith Goes to Washington (1939),Drama,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1208094,1247,4.395746,1,95,ALS,"Graduate, The (1967)",Comedy|Drama|Romance,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1208095,3578,4.395721,1,96,ALS,Gladiator (2000),Action|Adventure|Drama,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1208096,1252,4.394843,1,97,ALS,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1208097,47,4.392999,1,98,ALS,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1208098,2357,4.391974,1,99,ALS,Central Station (Central do Brasil) (1998),Drama,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1208099,1284,4.391594,1,100,ALS,"Big Sleep, The (1946)",Crime|Film-Noir|Mystery,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
providers = list(all_recs_joined.iloc[:,8:])
items_n = all_recs_joined.item.nunique()
proItems_n =  all_recs_joined.loc[ all_recs_joined['Drama'] == 1].item.nunique()
proItems_n_horror =  all_recs_joined.loc[ all_recs_joined['Horror'] == 1].item.nunique()

In [15]:
rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
rla.add_metric("rND")
rla.add_metric("rKL")
rla.add_metric("rRD")
rla.add_metric("div")
rla.add_metric("dem_parity")
results = rla.compute(all_recs_joined, test_data, "Drama", providers, items_n, proItems_n)
results.groupby('Algorithm').mean()

C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


,rND,rKL,rRD,div,dem_parity
Algorithm,,,,,
ALS,0.227811,0.113318,0.038723,0.929836,1.395804
FunkSVD,0.135141,0.050013,0.128865,0.879653,0.945063
ItemItem,0.184704,0.081687,0.079945,0.914770,1.227389
UserUser,0.236198,0.159207,0.555089,0.929723,0.664070


In [20]:
results

rND       rKL       rRD       div  dem_parity
user Algorithm                                                    
1    ALS        0.264105  0.112976  0.000000  0.947368    1.563234
     FunkSVD    0.092557  0.023621  0.052150  0.947368    0.945018
     ItemItem   0.266164  0.111351  0.000000  0.842105    1.389168
     UserUser   0.136502  0.046413  0.380717  0.894737    0.726944
2    ALS        0.275408  0.120079  0.000000  0.947368    1.712928
     FunkSVD    0.078330  0.020614  0.067342  0.947368    0.919127
     ItemItem   0.149131  0.042904  0.000000  1.000000    1.189324
     UserUser   0.205810  0.128165  0.512564  1.000000    0.683529
3    ALS        0.238019  0.101359  0.000000  1.000000    1.516972
     FunkSVD    0.082519  0.022199  0.071131  0.789474    0.852096
     ItemItem   0.203916  0.074967  0.124263  0.894737    1.309655
     UserUser   0.356981  0.308830  0.827797  1.000000    0.521452
4    ALS        0.488403  0.403037  0.000000  0.894737    2.075975
     FunkSVD    0.290033  0.176188  0.000000  0.894737    1.333502
     ItemItem   0.153691  0.047469  0.124263  0.894737    1.288112
     UserUser   0.218357  0.089074  0.594938  0.947368    0.589357
5    ALS        0.127149  0.032276  0.373080  0.894737    0.719234
     FunkSVD    0.101835  0.020451  0.055438  0.947368    1.097807
     ItemItem   0.188146  0.085190  0.210377  0.894737    0.995183
     UserUser   0.227283  0.123590  0.581950  0.947368    0.699233
6    ALS        0.089021  0.012755  0.000000  0.947368    1.009838
     FunkSVD    0.084641  0.023353  0.116904  0.789474    0.796804
     ItemItem   0.131550  0.030094  0.000000  0.894737    1.018699
     UserUser   0.290902  0.173897  0.734684  0.947368    0.563257
7    ALS        0.138042  0.031939  0.050309  0.947368    1.156284
     FunkSVD    0.157457  0.040800  0.000000  0.947368    1.264016
     ItemItem   0.091489  0.022710  0.070779  0.947368    1.332382
     UserUser   0.339209  0.219074  0.840532  1.000000    0.495646
8    ALS        0.080374  0.013527  0.031775  0.947368    1.139437
     FunkSVD    0.154915  0.041257  0.450997  0.894737    0.658938
...                  ...       ...       ...       ...         ...
6033 ItemItem   0.266389  0.111496  0.000000  0.947368    1.651198
     UserUser   0.292208  0.239102  0.695849  1.000000    0.612999
6034 ALS        0.292461  0.136986  0.000000  0.947368    1.595040
     FunkSVD    0.154334  0.042357  0.000000  0.842105    1.180367
     ItemItem   0.092666  0.017612  0.153584  0.842105    0.910600
     UserUser   0.360870  0.299955  0.848682  0.947368    0.502773
6035 ALS        0.219862  0.085552  0.119134  0.894737    1.381688
     FunkSVD    0.137158  0.073741  0.239565  0.947368    0.730737
     ItemItem   0.290543  0.135899  0.119134  0.894737    1.294224
     UserUser   0.155520  0.046872  0.158758  0.947368    0.985354
6036 ALS        0.379044  0.261850  0.000000  0.894737    1.514837
     FunkSVD    0.415233  0.288986  0.000000  0.947368    2.024604
     ItemItem   0.228353  0.082810  0.000000  0.894737    1.369284
     UserUser   0.188688  0.066917  0.055438  0.947368    1.169118
6037 ALS        0.186778  0.061055  0.000000  0.947368    1.512543
     FunkSVD    0.115917  0.030083  0.228029  0.947368    0.801437
     ItemItem   0.111623  0.027322  0.000000  0.947368    1.000463
     UserUser   0.327539  0.341975  0.764662  0.894737    0.547307
6038 ALS        0.242896  0.095471  0.000000  0.894737    1.585976
     FunkSVD    0.159839  0.047619  0.000000  0.894737    0.974712
     ItemItem   0.109759  0.025020  0.005129  0.842105    1.356004
     UserUser   0.373350  0.271775  0.896898  0.894737    0.481080
6039 ALS        0.328535  0.170784  0.000000  0.894737    1.845099
     FunkSVD    0.102210  0.021838  0.265278  0.894737    0.822207
     ItemItem   0.262747  0.109444  0.000000  0.894737    1.640413
     UserUser   0.467788  0.530572  1.024673  0.894737    0.501218
6040 ALS        0.322391  0.199195  0.000000  0.947368    1.685129
 

In [16]:
rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
rla.add_metric("rND")
rla.add_metric("rKL")
rla.add_metric("rRD")
rla.add_metric("div")
rla.add_metric("dem_parity")
results_horror = rla.compute(all_recs_joined, test_data, "Horror", providers, items_n, proItems_n_horror)
results_horror.groupby('Algorithm').mean()

,rND,rKL,rRD,div,dem_parity
Algorithm,,,,,
ALS,0.088602,0.094266,0.090214,0.929836,0.021434
FunkSVD,0.099735,0.116161,0.104139,0.879653,0.014038
ItemItem,0.096223,0.109651,0.101225,0.914770,0.016801
UserUser,0.095798,0.108651,0.101619,0.929723,0.018867


In [17]:
allResults = []
allResults.append(results.groupby('Algorithm').mean())
allResults.append(results_horror.groupby('Algorithm').mean())

In [18]:
pd.concat(allResults, keys=['Drama', 'Horror'])

rND       rKL       rRD       div  dem_parity
       Algorithm                                                    
Drama  ALS        0.227811  0.113318  0.038723  0.929836    1.395804
       FunkSVD    0.135141  0.050013  0.128865  0.879653    0.945063
       ItemItem   0.184704  0.081687  0.079945  0.914770    1.227389
       UserUser   0.236198  0.159207  0.555089  0.929723    0.664070
Horror ALS        0.088602  0.094266  0.090214  0.929836    0.021434
       FunkSVD    0.099735  0.116161  0.104139  0.879653    0.014038
       ItemItem   0.096223  0.109651  0.101225  0.914770    0.016801
       UserUser   0.095798  0.108651  0.101619  0.929723    0.018867